<a href="https://colab.research.google.com/github/Rahamatunnisa1121/ML-Algorithms/blob/main/Bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load Titanic dataset from seaborn for simplicity
import seaborn as sns
titanic = sns.load_dataset('titanic')

# View the dataset
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
!pip install numpy==1.24.4 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.1 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.


In [ ]:
# Select relevant columns and drop missing values
data = titanic[['sex', 'age', 'class', 'embarked', 'survived']].dropna()

In [ ]:
data.head()

,sex,age,class,embarked,survived
0,male,22.0,Third,S,0
1,female,38.0,First,C,1
2,female,26.0,Third,S,1
3,female,35.0,First,S,1
4,male,35.0,Third,S,0


In [ ]:
def categorize_age(age):
    if age < 18:
        return 'child'
    elif age < 60:
        return 'adult'
    else:
        return 'senior'
data['age'] = data['age'].apply(categorize_age)

In [ ]:
data.head()

,sex,age,class,embarked,survived
0,male,adult,Third,S,0
1,female,adult,First,C,1
2,female,adult,Third,S,1
3,female,adult,First,S,1
4,male,adult,Third,S,0


In [ ]:
# Convert all categorical variables to string type (important for pgmpy)
for col in data.columns:
    data[col] = data[col].astype(str)

data.head()

,sex,age,class,embarked,survived
0,male,adult,Third,S,0
1,female,adult,First,C,1
2,female,adult,Third,S,1
3,female,adult,First,S,1
4,male,adult,Third,S,0


In [ ]:
!pip install pgmpy==0.1.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: pgmpy
    Found existing installation: pgmpy 1.0.0
    Uninstalling pgmpy-1.0.0:
      Successfully uninstalled pgmpy-1.0.0


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator

# Define the structure (you can experiment here)
model = BayesianNetwork([
    ('sex', 'survived'),
    ('class', 'survived'),
    ('age', 'survived'),
    ('embarked', 'class')
])

model.fit(data, estimator=BayesianEstimator, prior_type='BDeu')  # or 'K2'


In [ ]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)

# Example: What's the chance of survival if person is female, adult, 1st class?
query = infer.query(
    variables=['survived'],
    evidence={'sex': 'female', 'age': 'adult', 'class': 'First'}
)

print(query)

+-------------+-----------------+
| survived    |   phi(survived) |
+=============+=================+
| survived(0) |          0.0292 |
+-------------+-----------------+
| survived(1) |          0.9708 |
+-------------+-----------------+


In [ ]:
#Model evaluation
from pgmpy.estimators import K2Score

score = K2Score(data)
print("Model K2 Score:", score.score(model))


Model K2 Score: -2385.1404389662857
